### Part B


#### Producer 2


In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import random
from datetime import datetime
import os
import pandas as pd

In [ ]:
# change directory (local machine only)
# os.chdir("A2")
# os.chdir("data")

# dirs = os.listdir(os.getcwd())
# print(dirs)

In [ ]:
hostip = "192.168.10.125"

In [ ]:
def random_time():
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
    return f"{hour:02}:{minute:02}:{second:02}"

In [ ]:
# reference code taken from FIT3182 applied session week 10
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding="utf-8")
        value_bytes = bytes(value, encoding="utf-8")
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print("Message published successfully. Data: " + str(value))
    except Exception as ex:
        print("Exception in publishing message.")
        print(str(ex))


# reference code taken from FIT3182 applied session week 10
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(
            bootstrap_servers=[f"{hostip}:9092"], api_version=(0, 10)
        )
    except Exception as ex:
        print("Exception while connecting Kafka.")
        print(str(ex))
    finally:
        return _producer


if __name__ == "__main__":
    topic = "hotspot_aqua"
    print("Publishing hotspot_AQUA_streaming records..")
    producer = connect_kafka_producer()

    data = pd.read_csv("hotspot_AQUA_streaming.csv")

    # feeding data loop
    while True:
        # randomly feed data (with replacement)
        row = data.sample(n=1).to_dict(orient="records")[0]

        # append producer information to identify the producer
        row["producer_id"] = "aqua_producer"

        # append randomly created time
        row["created_time"] = random_time()

        # call the publish_message function for encoding and sending the message
        publish_message(producer, topic, "parsed", dumps(row))

        # wait random n seconds seconds before proceeding
        sleep(random.uniform(0.5, 2))